In [1]:
print('hello world')

hello world


Outras similaridades: Cos, Ovl, Jacc, Dice

Semantic Similarity

These models find semantically similar sentences within one language or across languages:

**distiluse-base-multilingual-cased-v2**: Multilingual knowledge distilled version of multilingual Universal Sentence Encoder. While the original mUSE model only supports 16 languages, this multilingual knowledge distilled version supports 50+ languages.

**xlm-r-distilroberta-base-paraphrase-v1** - Multilingual version of distilroberta-base-paraphrase-v1, trained on parallel data for 50+ languages.

**xlm-r-bert-base-nli-stsb-mean-tokens**: Produces similar embeddings as the bert-base-nli-stsb-mean-token model. Trained on parallel data for 50+ languages.

**distilbert-multilingual-nli-stsb-quora-ranking** - Multilingual version of distilbert-base-nli-stsb-quora-ranking. Fine-tuned with parallel data for 50+ languages.

**T-Systems-onsite/cross-en-de-roberta-sentence-transformer** - Multilingual model for English an German. [More]

# Imports e métodos necessários

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline

In [3]:
import pandas as pd 
import xml.etree.ElementTree as et 

from scipy.stats import pearsonr

def parse_xml(xml_file):
    """Parse xml to pandas dataframe."""
    xtree = et.parse(xml_file)
    xroot = xtree.getroot() 

    df_cols = ['id', 't', 'h', 'similarity']
    rows = []

    for node in xroot:
        id_ = node.attrib.get("id")
        similarity = node.attrib.get("similarity")
        t = node.find("t").text
        h = node.find("h").text

        rows.append({
            "id": id_,
            "t": t, 
            "h": h,
            "similarity": similarity
        })
    return pd.DataFrame(rows, columns=df_cols, dtype=float)

def eval_similarity(pairs_gold, pairs_sys):
    '''
    Evaluate the semantic similarity output of the system against a gold score. 
    Results are printed to stdout.
    '''
    
    gold_values = np.array(pairs_gold)
    sys_values = np.array(pairs_sys)
    pearson = pearsonr(gold_values, sys_values)[0]
    absolute_diff = gold_values - sys_values
    mse = (absolute_diff ** 2).mean()
    
    print()
    print('Similarity evaluation')
    print('Pearson\t\tMean Squared Error')
    print('-------\t\t------------------')
    print('{:7.3f}\t\t{:18.2f}'.format(pearson, mse))

# Carregando os dados

In [4]:
!ls ../data/assin

assin-ptbr-dev.xml   assin-ptbr-train.xml  assin-ptpt-test.xml
assin-ptbr-test.xml  assin-ptpt-dev.xml    assin-ptpt-train.xml


In [5]:
df_ptbr_train = parse_xml('../data/assin/assin-ptbr-train.xml')
df_ptbr_dev = parse_xml('../data/assin/assin-ptbr-dev.xml')
df_ptbr_test = parse_xml('../data/assin/assin-ptbr-test.xml')

df_ptpt_train = parse_xml('../data/assin/assin-ptpt-train.xml')
df_ptpt_dev = parse_xml('../data/assin/assin-ptpt-dev.xml')
df_ptpt_test = parse_xml('../data/assin/assin-ptpt-test.xml')

In [6]:
print(f'assin-ptbr-train: {df_ptbr_train.shape}')
print(f'assin-ptbr-dev: {df_ptbr_dev.shape}')
print(f'assin-ptbr-test: {df_ptbr_test.shape}')
print()
print(f'assin-ptpt-train: {df_ptpt_train.shape}')
print(f'assin-ptpt-dev: {df_ptpt_dev.shape}')
print(f'assin-ptpt-test: {df_ptpt_test.shape}')

assin-ptbr-train: (2500, 4)
assin-ptbr-dev: (500, 4)
assin-ptbr-test: (2000, 4)

assin-ptpt-train: (2500, 4)
assin-ptpt-dev: (500, 4)
assin-ptpt-test: (2000, 4)


In [7]:
df_ptbr_train.head()

,id,t,h,similarity
0,1.0,"A gente faz o aporte financeiro, é como se a e...",Fernando Moraes afirma que não tem vínculo com...,2.00
1,2.0,"Em 2013, a história de como Walt Disney conven...",P.L.Travers era completamente contra a adaptaç...,2.25
2,3.0,"David Silva bateu escanteio, Kompany escalou a...","David Silva cobrou escanteio, o zagueiro se ap...",3.75
3,4.0,"Para os ambientalistas, as metas anunciadas pe...","Dilma aproveitou seu discurso ontem, na Confer...",2.75
4,5.0,"De acordo com a PM, por volta das 10h30 havia ...",O protesto encerrou por volta de 12h15 (horári...,2.00


## Fine-tuning Sentence-BERT

https://www.sbert.net/docs/training/overview.html

https://github.com/UKPLab/sentence-transformers/blob/master/examples/training/sts/training_stsbenchmark_continue_training.py

### xlm-r-distilroberta-base-paraphrase-v1

In [8]:
"""
This example loads the pre-trained SentenceTransformer model 'bert-base-nli-mean-tokens' from the server.
It then fine-tunes this model for some epochs on the STS benchmark dataset.
Note: In this example, you must specify a SentenceTransformer model.
If you want to fine-tune a huggingface/transformers model like bert-base-uncased, see training_nli.py and training_stsbenchmark.py
"""
from torch.utils.data import DataLoader
import math
from sentence_transformers import SentenceTransformer,  SentencesDataset, LoggingHandler, losses, util, InputExample
from sentence_transformers.evaluation import EmbeddingSimilarityEvaluator
from datetime import datetime
import os
import csv
from sentence_transformers import models, SentenceTransformer



# Read the dataset
train_batch_size = 16
num_epochs = 4
model_save_path = '../data/sbert_finetuning_assin_xlm-r-distilroberta-base-paraphrase-v1-comp-ptpt'# + datetime.now().strftime("%Y-%m-%d_%H-%M-%S")



# Load a pre-trained sentence transformer model
#model = SentenceTransformer(model_name)


model = SentenceTransformer('xlm-r-distilroberta-base-paraphrase-v1')

train_samples = []
dev_samples = []
#test_samples = []

for i, row in df_ptbr_train.iterrows():
    inp_example = InputExample(texts=[row['t'], row['h']], label=row['similarity'] / 5)
    train_samples.append(inp_example)
    
for i, row in df_ptbr_dev.iterrows():
    inp_example = InputExample(texts=[row['t'], row['h']], label=row['similarity'] / 5)
    dev_samples.append(inp_example)
    
#for i, row in df_ptbr_test.iterrows():
 #   inp_example = InputExample(texts=[row['t'], row['h']], label=row['similarity'] / 5)
  #  test_samples.append(inp_example)
    


train_dataset = SentencesDataset(train_samples, model)
train_dataloader = DataLoader(train_dataset, shuffle=True, batch_size=train_batch_size)
train_loss = losses.CosineSimilarityLoss(model=model)


evaluator = EmbeddingSimilarityEvaluator.from_input_examples(dev_samples, name='sts-dev')


# Configure the training. We skip evaluation in this example
warmup_steps = math.ceil(len(train_dataset) * num_epochs / train_batch_size * 0.1) #10% of train data for warm-up


# Train the model
model.fit(train_objectives=[(train_dataloader, train_loss)],
          evaluator=evaluator,
          epochs=num_epochs,
          evaluation_steps=1000,
          warmup_steps=warmup_steps,
          output_path=model_save_path)

In [9]:
t_embeddings = model.encode(df_ptbr_test['t'].tolist())
h_embeddings = model.encode(df_ptbr_test['h'].tolist())

In [10]:
from sklearn.metrics.pairwise import  cosine_similarity

similarities = [5.0 * cosine_similarity([t], [h])[0][0] for t, h in zip(t_embeddings, h_embeddings)]

In [11]:
pairs_gold = df_ptbr_test['similarity'].tolist()
pairs_sys = similarities

In [12]:
eval_similarity(pairs_gold, pairs_sys)


Similarity evaluation
Pearson		Mean Squared Error
-------		------------------
  0.792		              0.30


In [13]:
df_ptbr_test['predicted'] = similarities
df_ptbr_test.head()

,id,t,h,similarity,expected,predicted
0,1.0,"No Brasil, 809 instituições participarão da Pr...","Começa nesta segunda-feira, em todo o Brasil, ...",3.25,3.25,2.642748
1,2.0,Alguns dos novos participantes serão revelados...,Parte dos participantes será anunciada no Prog...,4.00,4.00,4.125273
2,3.0,"Nesta terça, o Inter revelou que o gringo sofr...","Nesta terça-feira, o departamento médico do cl...",4.50,4.50,3.798805
3,4.0,O texto da reforma política será agora encamin...,"Após passar pela Câmara, o texto tem que ir ai...",4.00,4.00,3.195306
4,5.0,O Fluminense vive crise em campo e não vence h...,"Nos últimos seis jogos, o Fluminense foi derro...",4.25,4.25,3.898855


In [45]:
from scipy.stats import spearmanr
print('Spearman correlation: {:7.3f}'.format(spearmanr(pairs_gold, pairs_sys)[0]))

Spearman correlation:   0.772


# Testes pt-pt

## Sentence-BERT

## Fine-tuning Sentence-BERT

https://www.sbert.net/docs/training/overview.html

https://github.com/UKPLab/sentence-transformers/blob/master/examples/training/sts/training_stsbenchmark_continue_training.py

### xlm-r-distilroberta-base-paraphrase-v1

In [14]:
"""
This example loads the pre-trained SentenceTransformer model 'bert-base-nli-mean-tokens' from the server.
It then fine-tunes this model for some epochs on the STS benchmark dataset.
Note: In this example, you must specify a SentenceTransformer model.
If you want to fine-tune a huggingface/transformers model like bert-base-uncased, see training_nli.py and training_stsbenchmark.py
"""
from torch.utils.data import DataLoader
import math
from sentence_transformers import SentenceTransformer,  SentencesDataset, LoggingHandler, losses, util, InputExample
from sentence_transformers.evaluation import EmbeddingSimilarityEvaluator
from datetime import datetime
import os
import csv
from sentence_transformers import models, SentenceTransformer



# Read the dataset
train_batch_size = 16
num_epochs = 4
model_save_path = '../data/sbert_ptpt_finetuning_assin_xlm-r-distilroberta-base-paraphrase-v1-comp'# + datetime.now().strftime("%Y-%m-%d_%H-%M-%S")



# Load a pre-trained sentence transformer model
#model = SentenceTransformer(model_name)



model_pt = SentenceTransformer('xlm-r-distilroberta-base-paraphrase-v1')

train_samples = []
dev_samples = []
#test_samples = []

for i, row in df_ptpt_train.iterrows():
    inp_example = InputExample(texts=[row['t'], row['h']], label=row['similarity'] / 5)
    train_samples.append(inp_example)
    
for i, row in df_ptpt_dev.iterrows():
    inp_example = InputExample(texts=[row['t'], row['h']], label=row['similarity'] / 5)
    dev_samples.append(inp_example)
    
#for i, row in df_ptpt_test.iterrows():
#    inp_example = InputExample(texts=[row['t'], row['h']], label=row['similarity'] / 5)
#    test_samples.append(inp_example)
    


train_dataset = SentencesDataset(train_samples, model_pt)
train_dataloader = DataLoader(train_dataset, shuffle=True, batch_size=train_batch_size)
train_loss = losses.CosineSimilarityLoss(model=model_pt)


evaluator = EmbeddingSimilarityEvaluator.from_input_examples(dev_samples, name='sts-dev')


# Configure the training. We skip evaluation in this example
warmup_steps = math.ceil(len(train_dataset) * num_epochs / train_batch_size * 0.1) #10% of train data for warm-up


# Train the model
model_pt.fit(train_objectives=[(train_dataloader, train_loss)],
          evaluator=evaluator,
          epochs=num_epochs,
          evaluation_steps=1000,
          warmup_steps=warmup_steps,
          output_path=model_save_path)

In [16]:
t_embeddings = model_pt.encode(df_ptpt_test['t'].tolist())
h_embeddings = model_pt.encode(df_ptpt_test['h'].tolist())

In [17]:
from sklearn.metrics.pairwise import  cosine_similarity

similarities = [5.0 * cosine_similarity([t], [h])[0][0] for t, h in zip(t_embeddings, h_embeddings)]

In [18]:
pairs_gold = df_ptpt_test['similarity'].tolist()
pairs_sys = similarities

In [19]:
eval_similarity(pairs_gold, pairs_sys)


Similarity evaluation
Pearson		Mean Squared Error
-------		------------------
  0.805		              0.49


In [20]:
df_ptpt_test['predicted'] = similarities
df_ptpt_test.head()

,id,t,h,similarity,predicted
0,1.0,Mau tempo abate-se sobre o Algarve e inunda a ...,A baixa de Albufeira foi afetada pelo mau tempo.,4.00,3.188667
1,2.0,"Em comparação com o ano anterior, registaram-s...",Feita a comparação com igual período do ano pa...,5.00,4.804944
2,3.0,Presidente da República fez uma série de aviso...,Foi naquele local que António Costa tomou poss...,1.75,2.693858
3,4.0,José Manuel da Silva reconheceu várias qualida...,"O novo ministro, Adalberto Campos Fernandes, g...",1.25,2.337724
4,5.0,Recorde-se que Mel Gibson esteve casado com Ro...,"Aos 60 anos, Mel Gibson volta a encontrar o amor.",2.00,2.632004


In [29]:
t_embeddings = model_pt.encode(df_ptbr_test['t'].tolist())
h_embeddings = model_pt.encode(df_ptbr_test['h'].tolist())
similarities = [5.0 * cosine_similarity([t], [h])[0][0] for t, h in zip(t_embeddings, h_embeddings)]
df_ptbr_test['predicted_pt'] = similarities
df_ptbr_test.head()

,id,t,h,similarity,predicted,predicted_pt
0,1.0,"No Brasil, 809 instituições participarão da Pr...","Começa nesta segunda-feira, em todo o Brasil, ...",3.25,2.642748,2.936442
1,2.0,Alguns dos novos participantes serão revelados...,Parte dos participantes será anunciada no Prog...,4.00,4.125273,3.999533
2,3.0,"Nesta terça, o Inter revelou que o gringo sofr...","Nesta terça-feira, o departamento médico do cl...",4.50,3.798805,3.715212
3,4.0,O texto da reforma política será agora encamin...,"Após passar pela Câmara, o texto tem que ir ai...",4.00,3.195306,3.451765
4,5.0,O Fluminense vive crise em campo e não vence h...,"Nos últimos seis jogos, o Fluminense foi derro...",4.25,3.898855,3.997440


In [30]:
t_embeddings = model.encode(df_ptpt_test['t'].tolist())
h_embeddings = model.encode(df_ptpt_test['h'].tolist())
similarities = [5.0 * cosine_similarity([t], [h])[0][0] for t, h in zip(t_embeddings, h_embeddings)]
df_ptpt_test['predicted_br'] = similarities
df_ptpt_test.head()

,id,t,h,similarity,predicted,predicted_br
0,1.0,Mau tempo abate-se sobre o Algarve e inunda a ...,A baixa de Albufeira foi afetada pelo mau tempo.,4.00,3.188667,3.704959
1,2.0,"Em comparação com o ano anterior, registaram-s...",Feita a comparação com igual período do ano pa...,5.00,4.804944,4.806061
2,3.0,Presidente da República fez uma série de aviso...,Foi naquele local que António Costa tomou poss...,1.75,2.693858,2.691873
3,4.0,José Manuel da Silva reconheceu várias qualida...,"O novo ministro, Adalberto Campos Fernandes, g...",1.25,2.337724,2.262819
4,5.0,Recorde-se que Mel Gibson esteve casado com Ro...,"Aos 60 anos, Mel Gibson volta a encontrar o amor.",2.00,2.632004,2.816572


In [31]:
df_ptpt_test.to_csv('ptpt_cruzado.csv', index=False)
df_ptbr_test.to_csv('ptbr_cruzado.csv', index=False)

In [41]:
print('Treinado PT-PT, avaliado PT-PT')
eval_similarity(df_ptpt_test['similarity'].tolist(), df_ptpt_test['predicted'].tolist())
print('\nTreinado PT-BR, avaliado PT-PT')
eval_similarity(df_ptpt_test['similarity'].tolist(), df_ptpt_test['predicted_br'].tolist())

Treinado PT-PT, avaliado PT-PT

Similarity evaluation
Pearson		Mean Squared Error
-------		------------------
  0.805		              0.49

Treinado PT-BR, avaliado PT-PT

Similarity evaluation
Pearson		Mean Squared Error
-------		------------------
  0.793		              0.53


In [37]:
pd.set_option('display.max_colwidth', None)
df_ptpt_test['diff'] = abs(df_ptpt_test['predicted'] - df_ptpt_test['predicted_br'])
df_ptpt_test.sort_values(by='diff', ascending=False).head(10)

,id,t,h,similarity,predicted,predicted_br,diff
1285,1286.0,Noruega e Roménia defrontam-se ainda esta quinta-feira no Pavilhão Municipal da Póvoa de Varzim.,Os bilhetes para os jogos que decorrerão no Pavilhão Muncipal da Póvoa do Varzim são gratuitos.,2.00,2.473669,1.458696,1.014974
1418,1419.0,"Nos últimos quatro anos, o aumento foi de cerca de metade desse valor.",Os portugueses não estão interessados em repetir a experiência dos últimos quatro anos.,1.75,2.009680,1.169388,0.840293
243,244.0,A companhia aérea brasileira Azul lança a 4 de maio o seu primeiro voo regular para a Europa.,O voo inaugural da Azul acontece já a 4 de maio.,4.00,3.856531,3.109473,0.747058
1502,1503.0,A Câmara da Figueira da Foz decretou hoje três dias de luto municipal na sequência do naufrágio de terça-feira.,"A barra da Figueira da Foz está fechada, na sequência de um naufrágio naquela zona.",3.50,1.614678,2.297439,0.682761
1318,1319.0,"O grupo VW anunciou, em 2014, um investimento de 677 milhões de euros para a modernização da VW Autoeuropa.","A fábrica Autoeuropa, do Grupo Volkswagen, anunciou o novo director geral.",1.00,2.273364,2.949896,0.676532
323,324.0,"A juntar-se ao New York City FC, Casillas iria reencontrar David Villa e Andoni Iraola, antigos companheiros de seleção.","O New York City FC está a preparar uma oferta para o guarda-redes do FC Porto, Iker Casillas.",1.75,2.828932,2.152556,0.676375
1766,1767.0,"A temperatura pode variar, hoje, entre 32 e 22 graus.","Ontem, a temperatura máxima na cidade chegou aos 33 graus, às 17 horas.",1.50,1.943733,2.589446,0.645714
1475,1476.0,O “Financial Times” foi vendido em 2015 e os jornalistas temem pelo futuro.,Os jornalistas do Financial Times anunciam que vão partir para a greve.,1.50,1.877116,2.516306,0.639190
868,869.0,Más notícias para Fábio Coentrão e para Leonardo Jardim.,Fábio Coentrão vai parar até final do ano.,1.50,1.761302,2.397497,0.636195
1901,1902.0,Justin Bieber desistiu oficialmente do romance com a cantora Selena Gomez para passar o réveillon ao lado de Hailey Baldwin.,Justin Bieber reencontrou novamente o amor ao lado de Hailey Baldwin.,3.00,3.684318,3.053008,0.631310


In [42]:
print('Treinado PT-BR, avaliado PT-BR')
eval_similarity(df_ptbr_test['similarity'].tolist(), df_ptbr_test['predicted'].tolist())
print('\nTreinado PT-PT, avaliado PT-BR')
eval_similarity(df_ptbr_test['similarity'].tolist(), df_ptbr_test['predicted_pt'].tolist())

Treinado PT-BR, avaliado PT-BR

Similarity evaluation
Pearson		Mean Squared Error
-------		------------------
  0.792		              0.30

Treinado PT-PT, avaliado PT-BR

Similarity evaluation
Pearson		Mean Squared Error
-------		------------------
  0.770		              0.33


In [38]:
pd.set_option('display.max_colwidth', None)
df_ptbr_test['diff'] = abs(df_ptbr_test['predicted'] - df_ptbr_test['predicted_pt'])
df_ptbr_test.sort_values(by='diff', ascending=False).head(10)

,id,t,h,similarity,predicted,predicted_pt,diff
531,532.0,O show está previsto para começar às 17h.,A atração acontece a partir das 12h.,1.75,1.534813,3.010281,1.475468
938,939.0,"Já para 2016, a previsão dos economistas recuou de 5,6% para 5,51%.","A mediana das estimativas passou de 5,76% para 5,71%.",2.25,2.213118,3.500387,1.287269
1320,1321.0,A defesa de Pimentel entrou com recurso e o processo está atualmente no Tribunal Superior Eleitoral (TSE).,Ainda não há decisão do TSE quanto ao recurso da defesa.,3.50,3.509004,2.293510,1.215494
591,592.0,"Ao todo, 323 municípios de todas as regiões poderão preencher 410 vagas ociosas.","Os profissionais selecionados preencherão vagas ociosas, principalmente por desistência.",2.50,1.487217,2.397150,0.909933
273,274.0,O “Oi 9º Dígito” serve para os sistemas Android e IOS e pode ser baixado gratuitamente.,O dígito 9 deve ser acrescentado à esquerda dos atuais números.,2.25,2.069565,1.285310,0.784255
742,743.0,"Quatro semanas atrás estava em R$ 3,10.","Quatro edições antes, estava em US$ 3,20 bilhões.",1.75,2.505740,3.280355,0.774615
839,840.0,A presidente anunciou ainda novos investimentos para a agricultura da região.,"Entre os anúncios, a presidente anunciou um investimento de R$ 700 milhões, de 2015 a 2018.",2.75,3.118390,2.383311,0.735079
395,396.0,"Na quarta-feira (22), o prêmio de R$ 3 milhões acumulou.",NInguém acertou as seis dezenas e o prêmio acumulou mais uma vez.,3.75,2.376354,1.661987,0.714367
1938,1939.0,Pessoa fez acordo de delação premiada com a Justiça em troca de redução de pena.,"Primeiro, para avaliar o impacto da delação premiada de Ricardo Pessoa.",2.75,2.550858,1.867840,0.683018
1109,1110.0,Uma das mudanças já definidas foi a instituição de gratificação para os conselheiros.,A gratificação foi instituída por um decreto de Dilma Rousseff.,3.50,2.895624,2.216990,0.678634
